In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model,feature_selection,preprocessing
from statsmodels.tools.eval_measures import mse
from statsmodels.tools.tools import add_constant
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import mean_absolute_percentage_error

import seaborn as sns
import matplotlib.pyplot as plt

from matplotlib.ticker import StrMethodFormatter
import scipy as imposp
from scipy import stats
import plotly.offline as po
import plotly.graph_objs as go
from numpy import median
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FormatStrFormatter

In [2]:
df=pd.read_csv('philly.csv')

In [3]:
df.shape

(506, 93)

In [4]:
df.head()

,livingAreaValue,zipcode,bathrooms,bedrooms,annualHomeownersInsurance,yearBuilt,resoFacts.yearBuiltEffective,price,monthlyHoaFee,solarPotential.sunScore,...,health_uninsured,veteran,resoFacts.architecturalStyle,resoFacts.subdivisionName,resoFacts.propertyCondition,State,citylabel_Philadelphia,income_household_median_bin_0-100k,income_household_median_bin_100k-200k,income_household_median_bin_200k-250k
0,943,19130,2,2,1155,1900.0,2018.8,275000,148,88.830,...,4.1,3.1,Unknown,Fairmount,Unknown,Pennsylvania,1,1,0,0
1,1230,19131,2,2,441,1960.0,2015.2,105000,893,66.460,...,7.3,5.7,Contemporary,Park Plaza,Unknown,Pennsylvania,1,1,0,0
2,746,19103,1,1,1155,1900.0,2004.0,275000,253,93.630,...,2.3,4.3,Straight Thru,Rittenhouse Square,Unknown,Pennsylvania,1,1,0,0
3,799,19106,1,1,1214,1880.0,2007.8,289000,465,91.890,...,3.5,3.0,Contemporary,Old City,Unknown,Pennsylvania,1,0,1,0
4,660,19106,1,1,1092,1900.0,2005.8,260000,213,87.008,...,3.5,3.0,Loft,Old City,Unknown,Pennsylvania,1,0,1,0


In [5]:
df['bathroom_bins']=pd.cut(df.bathrooms,bins = [0,1,2,3,4,5,6,12],labels=['1','2','3','4','5','6','6+'])
df['bedroom_bins']=pd.cut(df.bedrooms,bins = [0,1,2,3,4,5,6,8],labels=['1','2','3','4','5','6','6+'])
df['yearBuilt_bin']=pd.cut(df.yearBuilt,bins = [0,1980,2023],labels=['Pre-1980s','1980s+'])
#One hot encoding
df=pd.get_dummies(df, columns=['bathroom_bins','bedroom_bins','yearBuilt_bin','resoFacts.subdivisionName'
                              ])#, drop_first=True)

In [6]:
x=df.drop(['annualHomeownersInsurance','resoFacts.architecturalStyle',     
 'State','resoFacts.yearBuiltEffective','price','solarPotential.sunScore',
'resoFacts.hasGarage','resoFacts.hasView','resoFacts.isNewConstruction',
'resoFacts.taxAnnualAmount','listed_by.rating_average','resoFacts.pricePerSquareFoot',
'mortgage','population','age_under_10',
'age_10_to_19','age_20s','age_30s','age_40s','age_50s',
'age_60s','age_70s','age_over_80','male','female','divorced','widowed',
'family_size','family_dual_income','income_household_under_5','income_household_5_to_10',
'income_household_10_to_15','income_household_15_to_20','income_household_20_to_25','income_household_25_to_35',
'income_household_35_to_50','education_less_highschool','education_highschool','education_some_college',
'education_bachelors','education_graduate','education_college_or_above','education_stem_degree',
'labor_force_participation','unemployment_rate','farmer','race_white','race_black','race_asian',
'race_native','race_pacific','race_other','race_multiple','hispanic','disabled','poverty','limited_english',
'resoFacts.bathroomsHalf','health_uninsured','veteran','citylabel_Philadelphia','income_household_median_bin_200k-250k',
'resoFacts.waterViewYN','density',
'resoFacts.hasOpenParking','income_household_six_figure','income_household_100_to_150',
'resoFacts.hasAttachedGarage','income_household_50_to_75','commute_time',
'income_household_median',
           'income_household_75_to_100','income_household_150_over','income_individual_median',
           'married','never_married','home_ownership','income_household_median_bin_0-100k',
           'income_household_median_bin_100k-200k','age_median','resoFacts.bathroomsFull','resoFacts.daysOnZillow',
           'bedroom_bins_5','bedroom_bins_6','bedroom_bins_6+','bathroom_bins_6+',
           'bathroom_bins_6','yearBuilt_bin_Pre-1980s','transitScore.transit_score','zipcode',
           'bathroom_bins_3','bathroom_bins_4','bathroom_bins_5','yearBuilt','elementary school score',
           'resoFacts.propertyCondition',
'resoFacts.subdivisionName_Brewerytown',
'resoFacts.subdivisionName_Chestnut Hill', 
'resoFacts.subdivisionName_Clarendon Court',    
'resoFacts.subdivisionName_Dockside Condominium',
'resoFacts.subdivisionName_East Kensington',
'resoFacts.subdivisionName_Fairmount', 
'resoFacts.subdivisionName_Fitler Square',
'resoFacts.subdivisionName_Francisville',         
 'resoFacts.subdivisionName_Green Tree Summit',  
'resoFacts.subdivisionName_Hawthorne',           
'resoFacts.subdivisionName_Hopkinson House',  
'resoFacts.subdivisionName_Kensington',   
 'resoFacts.subdivisionName_Loft District', 
'resoFacts.subdivisionName_Mt Airy',   
'resoFacts.subdivisionName_Mt Airy (west)', 
'resoFacts.subdivisionName_Naval Square', 
 'resoFacts.subdivisionName_Newbold',      
'resoFacts.subdivisionName_None Available',
'resoFacts.subdivisionName_Norris Square', 
 'resoFacts.subdivisionName_Old Kensington',  
'resoFacts.subdivisionName_Olde Kensington', 
'resoFacts.subdivisionName_Park Plaza',         
'resoFacts.subdivisionName_Penn Sq',
'resoFacts.subdivisionName_River Park', 
'resoFacts.subdivisionName_Roxborough',   
'resoFacts.subdivisionName_Somerton',
'resoFacts.subdivisionName_South Kensington',
'resoFacts.subdivisionName_Unknown',
'resoFacts.subdivisionName_Waterfront Square',    
'resoFacts.subdivisionName_Wynnefield Hgts'      
                                      
                                      ], axis=1)
y=df[['price']]
y.head()

,price
0,275000
1,105000
2,275000
3,289000
4,260000


In [7]:
x.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 39 columns):
 #   Column                                        Non-Null Count  Dtype
---  ------                                        --------------  -----
 0   livingAreaValue                               506 non-null    int64
 1   bathrooms                                     506 non-null    int64
 2   bedrooms                                      506 non-null    int64
 3   monthlyHoaFee                                 506 non-null    int64
 4   walkScore.walkscore                           506 non-null    int64
 5   bathroom_bins_1                               506 non-null    uint8
 6   bathroom_bins_2                               506 non-null    uint8
 7   bedroom_bins_1                                506 non-null    uint8
 8   bedroom_bins_2                                506 non-null    uint8
 9   bedroom_bins_3                                506 non-null    uint8
 10  bedroom_bins_4

In [8]:
x.head()

,livingAreaValue,bathrooms,bedrooms,monthlyHoaFee,walkScore.walkscore,bathroom_bins_1,bathroom_bins_2,bedroom_bins_1,bedroom_bins_2,bedroom_bins_3,...,resoFacts.subdivisionName_Phila (south),resoFacts.subdivisionName_Philadelphia,resoFacts.subdivisionName_Point Breeze,resoFacts.subdivisionName_Queen Village,resoFacts.subdivisionName_Rittenhouse Square,resoFacts.subdivisionName_Society Hill,resoFacts.subdivisionName_Spring Gdn,resoFacts.subdivisionName_Wash Sq West,resoFacts.subdivisionName_Washington Sq,resoFacts.subdivisionName_Washington Sq West
0,943,2,2,148,96,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1230,2,2,893,48,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,746,1,1,253,99,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,799,1,1,465,98,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,660,1,1,213,99,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42)
linearmodel1 = sm.OLS(y_train, add_constant(x_train)).fit()
print(linearmodel1.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.834
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     49.84
Date:                Mon, 13 Jun 2022   Prob (F-statistic):          3.28e-120
Time:                        17:43:43   Log-Likelihood:                -5690.4
No. Observations:                 404   AIC:                         1.146e+04
Df Residuals:                     366   BIC:                         1.161e+04
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [10]:
y_pred = linearmodel1.predict(add_constant(x_test))

In [11]:
#MAE
mean_absolute_error(y_test, y_pred)

246258.84345234284

In [12]:
#MAPE
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test, y_pred)

0.4409065931824701

In [13]:
#RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

377059.2944189597

In [14]:
#Plot test errors by features

In [15]:
y_test=y_test.reset_index()

In [16]:
y_test

,index,price
0,173,90000
1,274,450000
2,491,2554735
3,72,595000
4,452,1750000
...,...,...
97,412,492000
98,436,215000
99,411,345000
100,86,615000


In [17]:
y_pred=y_pred.reset_index()

In [18]:
y_pred

,index,0
0,173,1.288343e+05
1,274,1.660261e+05
2,491,1.606303e+06
3,72,4.727728e+05
4,452,1.845338e+06
...,...,...
97,412,5.126395e+05
98,436,2.430089e+05
99,411,3.656242e+05
100,86,7.527798e+05


In [19]:
x_test=x_test.reset_index()

In [20]:
df2 = pd.merge(pd.merge(x_test,y_test,on='index'),y_pred,on='index')

In [21]:
df2['error']=abs(df2['price']-df2[0]).round()

In [22]:
df2['error']

0       38834.0
1      283974.0
2      948432.0
3      122227.0
4       95338.0
         ...   
97      20640.0
98      28009.0
99      20624.0
100    137780.0
101    150917.0
Name: error, Length: 102, dtype: float64

In [23]:
#sort absolute value of error descending
pd.set_option('display.max_columns', None)
df2=df2.sort_values(by=['error'],ascending=False)

In [24]:
#Philadelphia Undervalued Properties - less than 325k- evaluate errors (price drop of first error to $320k)
df2[(df2.price<=325000) & (df2.price<df2[0])].sort_values(by=['error'],ascending=False)

,index,livingAreaValue,bathrooms,bedrooms,monthlyHoaFee,walkScore.walkscore,bathroom_bins_1,bathroom_bins_2,bedroom_bins_1,bedroom_bins_2,bedroom_bins_3,bedroom_bins_4,yearBuilt_bin_1980s+,resoFacts.subdivisionName_Academy House,resoFacts.subdivisionName_Art Museum,resoFacts.subdivisionName_Art Museum (fairmo,resoFacts.subdivisionName_Art Museum (spring,resoFacts.subdivisionName_Art Museum Area,resoFacts.subdivisionName_Avenue Of The Arts,resoFacts.subdivisionName_Bella Vista,resoFacts.subdivisionName_Center City,resoFacts.subdivisionName_Chinatown,resoFacts.subdivisionName_East Falls,resoFacts.subdivisionName_Fishtown,resoFacts.subdivisionName_Graduate Hospital,resoFacts.subdivisionName_Logan Square,resoFacts.subdivisionName_Northern Liberties,resoFacts.subdivisionName_Old City,resoFacts.subdivisionName_Olde City,resoFacts.subdivisionName_Penns Landing,resoFacts.subdivisionName_Phila (south),resoFacts.subdivisionName_Philadelphia,resoFacts.subdivisionName_Point Breeze,resoFacts.subdivisionName_Queen Village,resoFacts.subdivisionName_Rittenhouse Square,resoFacts.subdivisionName_Society Hill,resoFacts.subdivisionName_Spring Gdn,resoFacts.subdivisionName_Wash Sq West,resoFacts.subdivisionName_Washington Sq,resoFacts.subdivisionName_Washington Sq West,price,0,error
35,153,1769,2,3,322,91,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,330000,800524.525962,470525.0
38,208,880,1,1,613,99,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,300000,556685.162173,256685.0
13,78,889,1,2,689,99,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,268500,498899.103640,230399.0
10,218,435,1,0,336,91,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,119000,291720.603536,172721.0
80,69,1010,1,1,274,93,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269000,428867.271616,159867.0
60,63,1867,2,3,1168,48,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,315000,453689.712242,138690.0
84,312,678,1,1,504,99,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,175000,300668.687896,125669.0
28,210,1196,1,2,783,91,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,274900,396177.643086,121278.0
53,101,1001,2,1,923,99,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,319900,389365.325556,69465.0
37,336,444,1,0,379,99,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,154900,222346.310386,67446.0


In [25]:
#Philadelphia Overvalued Properties - less than 325k- evaluate errors
df2[(df2.price<=325000) & (df2.price>df2[0])].sort_values(by=['error'],ascending=False)

,index,livingAreaValue,bathrooms,bedrooms,monthlyHoaFee,walkScore.walkscore,bathroom_bins_1,bathroom_bins_2,bedroom_bins_1,bedroom_bins_2,bedroom_bins_3,bedroom_bins_4,yearBuilt_bin_1980s+,resoFacts.subdivisionName_Academy House,resoFacts.subdivisionName_Art Museum,resoFacts.subdivisionName_Art Museum (fairmo,resoFacts.subdivisionName_Art Museum (spring,resoFacts.subdivisionName_Art Museum Area,resoFacts.subdivisionName_Avenue Of The Arts,resoFacts.subdivisionName_Bella Vista,resoFacts.subdivisionName_Center City,resoFacts.subdivisionName_Chinatown,resoFacts.subdivisionName_East Falls,resoFacts.subdivisionName_Fishtown,resoFacts.subdivisionName_Graduate Hospital,resoFacts.subdivisionName_Logan Square,resoFacts.subdivisionName_Northern Liberties,resoFacts.subdivisionName_Old City,resoFacts.subdivisionName_Olde City,resoFacts.subdivisionName_Penns Landing,resoFacts.subdivisionName_Phila (south),resoFacts.subdivisionName_Philadelphia,resoFacts.subdivisionName_Point Breeze,resoFacts.subdivisionName_Queen Village,resoFacts.subdivisionName_Rittenhouse Square,resoFacts.subdivisionName_Society Hill,resoFacts.subdivisionName_Spring Gdn,resoFacts.subdivisionName_Wash Sq West,resoFacts.subdivisionName_Washington Sq,resoFacts.subdivisionName_Washington Sq West,price,0,error
67,82,600,1,0,0,99,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,216900,-237917.507172,454818.0
16,124,924,2,1,298,99,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,289500,11197.494116,278303.0
14,323,567,1,1,0,99,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,204900,-64921.211884,269821.0
20,271,631,1,1,345,91,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,160000,-103779.488242,263779.0
49,0,943,2,2,148,96,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,275000,43312.619861,231687.0
76,104,509,1,1,175,97,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,189900,-15830.229017,205730.0
11,9,1251,2,2,758,54,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,220000,55970.606071,164029.0
70,46,1037,1,2,259,35,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,169000,61141.955220,107858.0
54,268,567,1,1,443,99,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,210000,110664.890178,99335.0
63,18,531,1,1,387,100,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,265000,204921.254293,60079.0
